In [1]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [2]:
import sys
sys.path.append("..")
import torch
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="whitegrid", color_codes=True)
import train
import train_model_no_WS
from helpers.train_helpers import get_dataset, get_device, accuracy

%matplotlib inline

In [3]:
torch.cuda.is_available()

True

In [4]:
%load_ext tensorboard
%tensorboard --logdir ./logs

config_model_no_WS = dict(
    logs_dir=f'./logs/no_ws/{datetime.now().strftime("%Y%m%d_%H%M%S")}',
    optimizer='Adam',
    optimizer_learning_rate=0.001,
    batch_size=128,
    num_epochs=80,  # 250
    model='simple_conv',
    class_num=10,
    channels_in=1,
    augmentation=False,
    verbose=0
)

model = train_model_no_WS.train(config_model_no_WS)

In [5]:
config = dict(batch_size=1, augmentation=False)
training_loader, test_loader = get_dataset(config)
device = get_device()
criterion = torch.nn.CrossEntropyLoss()

In [9]:
for batch_x, batch_y, batch_classes in test_loader:
    print(batch_x_1.shape)
    batch_x_1 = batch_x[:, 0, :, :].view(-1, 1, batch_x.size(2), batch_x.size(3)).to(device)
    batch_x_2 = batch_x[:, 1, :, :].view(-1, 1, batch_x.size(2), batch_x.size(3)).to(device)

    batch_classes_1 = batch_classes[:, 0].to(device)
    batch_classes_2 = batch_classes[:, 1].to(device)
    print(batch_classes_1.shape)
    print(batch_classes_2.shape)

    print(batch_y.shape)
    batch_y = batch_y.to(device)

    prediction_1 = model(batch_x_1)
    prediction_2 = model(batch_x_2)
    print(prediction_1)
    print(prediction_2)
    loss = (criterion(prediction_1, batch_classes_1) + criterion(prediction_2, batch_classes_2)) / 2
    print("loss=", loss)
    acc = accuracy(prediction_1.argmax(1) <= prediction_2.argmax(1), batch_y, argmax=False)
    print("acc=", acc)
    break

torch.Size([1, 1, 28, 28])
torch.Size([1])
torch.Size([1])
torch.Size([1])
tensor([[ -3.6193,  -9.2546,  42.6293,  10.5178,  -9.3457, -13.9523, -19.2700,
          -6.8574,  -1.1432, -16.5079]], device='cuda:0',
       grad_fn=<AddmmBackward>)
tensor([[-19.8581,  -8.3171, -21.7095,  -2.8992,  33.5805, -12.0219, -18.1853,
          13.2548,   2.2654,  10.2091]], device='cuda:0',
       grad_fn=<AddmmBackward>)
loss= tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
acc= tensor(1., device='cuda:0')
